In [1]:
# Data loading and manipulation:
using HDF5, JLD
include("../src/tmp_inst_rts96.jl")

# Analysis:
include("../src/TemporalInstanton.jl")
using TemporalInstanton

In [2]:
# Plotting:
using Interact
using PyPlot
#PyPlot.svg(true)

    call(Type{PyCall.PyDict}, Base.Associative{#K<:Any, Any}) at /home/jkersulis/.julia/v0.4/PyCall/src/conversions.jl:464
is ambiguous with: 
    call(Type{PyCall.PyDict}, Base.Associative{Any, #V<:Any}) at /home/jkersulis/.julia/v0.4/PyCall/src/conversions.jl:463.
To fix, define 
    call(Type{PyCall.PyDict}, Base.Associative{Any, Any})
before the new definition.


LoadError: LoadError: LoadError: LoadError: TypeError: apply_type: in first parameter of NTuple, expected Int64, got Type{Type{T}}
while loading /home/jkersulis/.julia/v0.4/PyCall/src/PyCall.jl, in expression starting on line 381
while loading /home/jkersulis/.julia/v0.4/PyPlot/src/PyPlot.jl, in expression starting on line 3
while loading In[2], in expression starting on line 3

In [3]:
using MatpowerCases
mpc = loadcase("case96")

Dict{ASCIIString,Any} with 7 entries:
  "baseMVA" => 100.0
  "branch"  => 120x11 Array{Float64,2}:…
  "areas"   => 6x2 Array{Float64,2}:…
  "gen"     => 99x10 Array{Float64,2}:…
  "bus"     => 73x13 Array{Float64,2}:…
  "gencost" => 99x7 Array{Float64,2}:…
  "version" => '1'

In [81]:
mpc["bus"][:,10]

73-element Array{Float64,1}:
 138.0
 138.0
 138.0
 138.0
 138.0
 138.0
 138.0
 138.0
 138.0
 138.0
 230.0
 230.0
 230.0
   ⋮  
 230.0
 230.0
 230.0
 230.0
 230.0
 230.0
 230.0
 230.0
 230.0
 230.0
 230.0
 230.0

In [6]:
####### LOAD DATA ########
psData = psDataLoad()

# unpack psDL (boilerplate):
(Sb,f,t,r,x,b,Y,bustype,
Gp,Gq,Dp,Dq,Rp,Rq,
Pmax,Pmin,Qmax,Qmin,Plim,
Vg,Vceiling,Vfloor,
busIdx,N,Nr,Ng,k) = unpack_psDL(psData)

Sb = 100e6 #overwrite "100.0"

res = r
reac = x

####### LINK DATA ########
# Static
Ridx = find(Rp) # Vector of renewable nodes
Y = full(Y) # Full admittance matrix (ref not removed)
ref = 1 # Index of ref node
k = k # Conventional generator participation factors
lines = [(f[i],t[i]) for i in 1:length(f)];
line_lengths = load("../data/RTS-96\ Data/line_lengths.jld", "line_lengths")

# Thermal model parameters:
Tamb = 35. # C
T0 = 60. #46. # initial line steady-state temp
time_intervals = 3 # thirty minutes
time_values = 0:60:600 # ten minutes in 1-min steps
int_length = 600. # seconds

println("loaded")

loaded


In [7]:
# time_intervals = 3 # thirty minutes
# time_values = 0:60:600 # ten minutes in 1-min steps
# int_length = 600. # seconds

time_intervals = 6 # thirty minutes total
time_values = 0:30:300 # five minutes in 30-sec steps
int_length = 300. # seconds = 5 min

# Generation, demand, and wind gen forecast:
G0 = [0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp]
D0 = [0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp]
P0 = [Rp;1.1*Rp;1.2*Rp;1.3*Rp;1.4*Rp;1.5*Rp]

# G0 = [0.7*Gp;0.7*Gp;0.7*Gp]
# D0 = [0.9*Dp;0.9*Dp;0.9*Dp]
# P0 = [Rp;1.5*Rp;1.5*Rp]

# Temporal instanton analysis:
# score,x,θ,α,diffs,xopt = solve_temporal_instanton(
#     Ridx,Y,G0,P0,D0,Sb,ref,lines,
#     res,reac,k,line_lengths,Tamb,
#     T0,int_length)
# println("Unsolved lines: $(sum(score.==Inf))")

438-element Array{Float64,1}:
 0.0      
 0.0820657
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 ⋮        
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      
 0.0      

In [8]:
    n = length(k)
    nr = length(Ridx)
    T = round(Int64,length(find(P0))/nr)
    numLines = 1#length(lines)

    # Vars used to store results:
    score = Float64[]
    α = Array(Vector{Float64},0)
    θ = Array(Array,0)
    x = Array(Array,0)
    diffs = Array(Array,0)
    xopt = Array(Array,0)

    # Form objective quadratic:
    Qobj = tmp_inst_Qobj(n,nr,T; pad=true)
    G_of_x = (Qobj,0,0)

    # Create A1 (only A2, the bottom part,
    # changes during line loop):
    A1 = tmp_inst_A1(Ridx,T,Y,ref,k; pad=true)

    b = tmp_inst_b(n,T,G0,P0,D0; pad=true)
    Qtheta = tmp_inst_Qtheta(n,nr,T)

    # parallelize the loop:
    # addprocs(3)

    # Loop through all lines:
idx = 1
line = lines[idx]
line_model = LineModel(line[1],
            line[2],
            res[idx],
            reac[idx],
            line_lengths[idx],
            NaN,
            NaN,
            NaN,
            NaN,
            NaN,
            NaN,
            NaN,
            NaN)

add_thermal_parameters(line_model, "waxwing")

therm_a = compute_a(line_model.mCp,
                    line_model.ηc,
                    line_model.ηr,
                    Tamb,
                    line_model.Tlim)
therm_c = compute_c(line_model.mCp,
                    line_model.rij,
                    line_model.xij,
                    Sb,
                    line_model.length)
therm_d = compute_d(line_model.mCp,
                    line_model.ηc,
                    line_model.ηr,
                    Tamb,
                    line_model.Tlim,
                    line_model.qs)
therm_f = compute_f(int_length,
                    therm_a,
                    therm_d,
                    T,
                    T0)

# thermal constraint, Q(z) = 0:
kQtheta = (therm_a/therm_c)*(line_model.Tlim - therm_f)
Q_of_x = (Qtheta,0,kQtheta)

# array of vectors with Float64 values:
deviations = Array(Vector{Float64},0)
angles = Array(Vector{Float64},0)
alpha = Float64[]

# Create A2 based on chosen line:
A2 = tmp_inst_A_scale_new(n,Ridx,T,line,therm_a,int_length)
# Stack A1 and A2:
A = [A1; A2]

# Computationally expensive part: solving QCQP
xvec,sol = solve_instanton_qcqp(G_of_x,Q_of_x,A,b,T)

push!(score,sol)
if isinf(sol)
    push!(deviations,[])
    push!(angles,[])
    push!(alpha,NaN)
    push!(diffs,[])
else
    # Variable breakdown:
    # (nr+n+1) per time step
    #   First nr are deviations
    #   Next n are angles
    #   Last is mismatch
    # T variables at the end: anglediffs
    for t = 1:T
        push!(deviations,xvec[(nr+n+1)*(t-1)+1:(nr+n+1)*(t-1)+nr])
        push!(angles,xvec[(nr+n+1)*(t-1)+nr+1:(nr+n+1)*(t-1)+nr+n])
        push!(alpha,xvec[(nr+n+1)*(t)])
    end
    push!(diffs,xvec[end-T+1:end])
end

push!(x,deviations)
push!(θ,angles)
push!(α,alpha)
push!(xopt,xvec)
# shut down procs (parallelization over)
#rmprocs([2,3,4])
# return score,x,θ,α,diffs,xopt

1-element Array{Array{T,N},1}:
 [-0.00103457,-6.1812e-7,2.49945e-5,2.8508e-5,3.01445e-5,1.42095e-5,4.83123e-6,3.16641e-5,-3.26807e-7,-7.91578e-6  …  0.447178,0.33624,0.455489,-17.1078,7.2576e-5,0.00011857,0.000197684,0.00034967,0.000758237,0.0146715]

In [6]:
addprocs(3)

3-element Array{Any,1}:
 2
 3
 4

In [7]:
nprocs()

4

In [16]:
workers()

3-element Array{Int64,1}:
 2
 3
 4

In [22]:
rmprocs(1)

:ok

In [23]:
workers()

1-element Array{Int64,1}:
 1

In [6]:
using IProfile

In [7]:
@iprofile report

  count  time(%)   time(s) bytes(%) bytes(k)
       2   0.00   0.000000   0.00         0  	# /home/jkersulis/Documents/Dropbox/InstantonAnew/Jonas Edits/Time-Coupled Instanton/temporal-instanton/src/TemporalInstanton3.jl, line 247, @mstr " Return A1, A2, A3 where:\n    * A1 corresponds to wind\n    * A2 corresponds to angles + mismatch\n    * A3 corresponds to angle difference vars\n\n    Used to find x_star, the min-norm solution to\n    Ax=b such that x_star[idx3] = 0.\n    "
       2   0.00   0.000000   0.00         0  	# line 255, (m,n) = size(A)
       2   0.00   0.000274   0.00        11  	# line 256, idx1 = find(diag(Qobj))
       2   0.01   0.000625   0.01        81  	# line 257, idx2 = setdiff(1:n - T,idx1)
       2   0.00   0.000000   0.00         0  	# line 258, idx3 = (n - T) + 1:n
       2   0.00   0.000056   0.00        11  	# line 259, idx = [idx1,idx2,idx3]
       2   0.00   0.000151   0.06       778  	# line 261, A1 = A[:,idx1]
       2   0.01   0.000596   0.24      31

The profiling results are clear: the most time-intensive operation by far is `find_x_star()`

In [19]:
function partition_A(A,Qobj,T)
    """ Return A1, A2, A3 where:
    * A1 corresponds to wind
    * A2 corresponds to angles + mismatch
    * A3 corresponds to angle difference vars

    Used to find x_star, the min-norm solution to
    Ax=b such that x_star[idx3] = 0.
    """
    m,n = size(A)   
    idx1 = find(diag(Qobj))
    idx2 = setdiff(1:n-T,idx1)
    idx3 = n-T+1:n
    idx = [idx1,idx2,idx3]
    
    A1 = A[:,idx1]
    A2 = A[:,idx2]
    A3 = A[:,idx3]
    return A1,A2,A3,idx1,idx2,idx3
end

function find_x_star(Qobj,A,b,T)
    """ x_star is the vector by which the problem must
    be translated in the first step of the temporal
    instanton QCQP solution.

    x_star is chosen to be the point in the set Ax=b
    nearest to the origin such that x_star[idx3] = 0.
    This condition ensures no linear term is introduced
    into the quadratic constraint.
    """
    A1,A2,A3,idx1,idx2,idx3 = partition_A(A,Qobj,T)
    x_star = zeros(size(A,2))
    x_star[[idx1,idx2]] = [A1 A2]\b
    return x_star
end

find_x_star (generic function with 1 method)

In [112]:
function find_x_star(Qobj,A,b,T)
    """ x_star is the vector by which the problem must
    be translated in the first step of the temporal
    instanton QCQP solution.

    x_star is chosen to be the point in the set Ax=b
    nearest to the origin such that x_star[idx3] = 0.
    This condition ensures no linear term is introduced
    into the quadratic constraint.
    """
    A1,A2,A3,idx1,idx2,idx3 = partition_A(A,Qobj,T)
    x_star = zeros(size(A,2))
    Z = sparse([A1 A2]')
    x_star[[idx1,idx2]] = (Z/(Z'*Z))*b
    return x_star
end

find_x_star (generic function with 1 method)

In [25]:
A1,A2,A3,idx1,idx2,idx3 = partition_A(A,Qobj,T)

(
450x108 Array{Float64,2}:
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 -1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  ⋮       

In [111]:
# MAJOR speedup
Z = sparse([A1 A2]')
@time (Z/(Z'*Z))*b
maxabs([A1 A2]\b - (Z/(Z'*Z))*b) # solutions are the same

elapsed time: 0.117862209 seconds (27030696 bytes allocated, 45.43% gc time)


552-element Array{FloatingPoint,1}:
 -0.000624546
 -0.895284   
 -0.953705   
 -0.973778   
 -0.979983   
 -0.939377   
 -0.924974   
 -0.983382   
 -0.915818   
 -0.849971   
 -0.897198   
 -0.905315   
 -0.934835   
  ⋮          
  0.196464   
  0.187866   
  0.231396   
  0.241256   
  0.176781   
  0.188566   
  0.258566   
  0.333242   
  0.203333   
  0.110962   
  0.203558   
 -1.05169    

## Translation

The following methods partition the $A$ matrix, find the appropriate translation point $x^*$, and translate a general quadratic:

In [ ]:
function partition_A(A,Qobj,T) # cheap
    """ Return A1, A2, A3 where:
    * A1 corresponds to wind
    * A2 corresponds to angles + mismatch
    * A3 corresponds to angle difference vars

    Used to find x_star, the min-norm solution to
    Ax=b such that x_star[idx3] = 0.
    """
    m,n = size(A)   
    idx1 = find(diag(Qobj))
    idx2 = setdiff(1:n-T,idx1)
    idx3 = n-T+1:n
    idx = [idx1,idx2,idx3]
    
    A1 = A[:,idx1]
    A2 = A[:,idx2]
    A3 = A[:,idx3]
    return A1,A2,A3,idx1,idx2,idx3
end

function find_x_star(Qobj,A,b,T) # expensive: matrix inversion
    """ x_star is the vector by which the problem must
    be translated in the first step of the temporal
    instanton QCQP solution.

    x_star is chosen to be the point in the set Ax=b
    nearest to the origin such that x_star[idx3] = 0.
    This condition ensures no linear term is introduced
    into the quadratic constraint.
    """
    A1,A2,A3,idx1,idx2,idx3 = partition_A(A,Qobj,T)
    x_star = zeros(size(A,2))
    x_star[[idx1,idx2]] = [A1 A2]\b # can we avoid this?
    return x_star
end

function translate_quadratic(G_of_x,x_star) # cheap
    """ This function performs the change of variables from x to z,
    where z = x - x_star. (For translating a quadratic problem.)
    Returns triple H_of_x consisting of matrix H, vector h, constant kh.

    Arguments
    G_of_x consists of matrix G, vector g, constant kg.
    x_star is translation.

    Used to perform second step of temporal instanton solution method,
    assuming x_star is min-norm solution of Ax=b.
    """
    G,g,kg = G_of_x
    if g == 0
        g = zeros(size(G,1),1)
    end
    H = G
    h = g + 2*G*x_star
    kh = kg + x_star'*G*x_star + g'*x_star
    return (H,h,kh[1])
end

## Kernel mapping

Here we "rotate" by $N^\top$, where $N$ consists of $k$ columns spanning $\mathcal{N}(A)$.

In [1]:
function kernel_rotation(A; dim_N_only=true)
    """ Find an orthonormal basis for the nullspace of A.
    This matrix may be used to rotate a temporal instanton
    problem instance to eliminate all but nullity(A) elements.
    """
    m,n = size(A)

    # Assume A always has full row rank of m.
    # It may be possible for this assumption to fail
    # due to numerics, but a rank() check is expensive.
    
    #if isposdef(A*A')
    dim_N = n - m # dimension of nullspace of A
    # else
    #     dim_N = n - rank(A)
    #     warn("A does not have full row rank.")
    # end
    q = qr(A'; thin=false)[1]
    # N = circshift(q,(0,dim_N))

    # Usually we only need the cols that span N(A):
    if dim_N_only
        return q[:,end-dim_N+1:end]
    else
        return N
    end
end

function rotate_quadratic(G_of_x,R) # cheap
    """ Rotate quadratic G_of_x by
    rotation matrix R.
    """
    G,g,kg = G_of_x
    return (R*G*R',R*g,kg)
end

rotate_quadratic (generic function with 1 method)

In [31]:
function kernel_rotation(A; spqr=false)
    """ Find an orthonormal basis for the nullspace of A.
    This matrix may be used to rotate a temporal instanton
    problem instance to eliminate all but nullity(A) elements.
    """
    m,n = size(A)

    # Assume A always has full row rank of m.
    # It may be possible for this assumption to fail
    # due to numerics, but a rank() check is expensive.
    dim_N = n - m # dimension of nullspace of A

    if spqr
        F = qrfact(sparse(A'))
        # B selects last dim_N cols of Q:
        B = [zeros(size(A,2)-dim_N,dim_N); eye(dim_N)]
        N = convert(Array,SparseMatrix.SPQR.qmult(SparseMatrix.SPQR.QX, F, SparseMatrix.CHOLMOD.Dense(B)))
        return N
    else
        q = qr(A'; thin=false)[1]
        return q[:,end-dim_N+1:end]
    end
end

kernel_rotation (generic function with 1 method)

In [2]:
function kernel_rotation_sparse(A)
    """ Find an orthonormal basis for the nullspace of A.
    This matrix may be used to rotate a temporal instanton
    problem instance to eliminate all but nullity(A) elements.
    """
    m,n = size(A)
    # Assume A always has full row rank of m.
    # It may be possible for this assumption to fail
    # due to numerics, but a rank() check is expensive.
    dim_N = n - m # dimension of nullspace of A
    
    # qr factorization of A using fill-reducing permutation:
    F = qrfact(sparse(A'))
    # B selects last dim_N cols of Q:
    B = [zeros(size(A,2)-dim_N,dim_N); eye(dim_N)]
    N = convert(Array,SparseMatrix.SPQR.qmult(SparseMatrix.SPQR.QX, F, SparseMatrix.CHOLMOD.Dense(B)))
end

kernel_rotation_sparse (generic function with 1 method)

In [36]:
# Save A matrix to make experimentation faster:
using HDF5,JLD
#save("../data/A.jld","A",A)
A = load("../data/A.jld")["A"]

450x558 Array{Float64,2}:
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0
 -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0
  ⋮         

In [38]:
A = randn(5,10)

5x10 Array{Float64,2}:
 -0.707836   -0.182463   -1.66661    …   0.612153  -1.45728     0.0184265
 -1.71253    -0.570917    0.962158      -1.3346     0.447239   -1.28922  
 -0.353017   -0.841213    1.42722        1.35516    1.05856    -0.905243 
 -0.0205593   1.16072    -0.242938       0.624634  -0.0362315   0.242769 
 -0.073743   -0.0475827  -0.0371604     -0.414996  -0.520215   -0.506054 

In [39]:
@time N1 = kernel_rotation(A,spqr=false);
@time N2 = kernel_rotation(A,spqr=true);

 138.312 microseconds (40 allocations: 5504 bytes)
 135.143 microseconds (26 allocations: 4704 bytes)


In [24]:
F = qrfact(sparse(A'))
dim_N = 108
B = [zeros(size(A,2)-dim_N,dim_N); eye(dim_N)]
N2 = SparseMatrix.SPQR.qmult(SparseMatrix.SPQR.QX, F, SparseMatrix.CHOLMOD.Dense(B))

558x108 Base.SparseMatrix.CHOLMOD.Dense{Float64}:
 -0.0657601   -0.0029995   -0.199876    …   0.0          0.0        
 -0.0154563   -0.0096515   -0.00981624      0.0          0.0        
 -0.0257904   -0.00898555   0.00144276      0.0          0.0        
 -0.0346138   -0.00846669   0.00510294      0.0          0.0        
 -0.0387583   -0.00825614   0.00695442      0.0          0.0        
 -0.00985561  -0.00983352   0.00529716  …   0.0          0.0        
  0.00404939  -0.0105823    0.00892111      0.0          0.0        
 -0.0426407   -0.00808774   0.00878667      0.0          0.0        
  0.0117509   -0.0109909    0.0110887       0.0          0.0        
 -0.164366    -0.0150967    0.156389        0.0          0.0        
  0.572317    -0.0687488    0.0589382   …   0.0          0.0        
  0.047082    -0.0491359   -0.131211        0.0          0.0        
 -0.141484     0.831288     0.109668        0.0          0.0        
  ⋮                                     ⋱            

Taking the rank of $A$ is expensive, even if I use Julia's built-in function. There must be a better way to determine the dimension of the null space.

If $A$ has full row rank, we can just take n - m to find the dimension of the null space. To determine whether $A$ has full row rank, we can check the rank of $AA^\top$.

Or we can assume $A$ has full row rank and skip the check (which is somewhat expensive itself) altogether. I think $A$ will always have full row rank unless the admittance matrix is singular or the auxiliary variable coeffients are non-unique. This should never be the case.

The most expensive thing remaining is the qr factorization of $A^\top$, which is needed to find a basis for the null space of $A$. A newer version of Julia (0.4) can exploit sparsity in qr factorizations, so when I update (which I am unsure how to do), I should be able to reduce computation time still more.

## Constraint Eigendecomposition and rotation

Here we change variables from $z$ to $w$ to render the constraint a norm constraint. This is done by letting $w=KU^\top z$ where `D,U = eig(Q_of_z[1])`. Thus, we can use the previously defined rotation method with $R=(UK^{-1})^\top$.

In [ ]:
function return_K(D)
    K = ones(length(D))
    K[find(D)] = sqrt(D[find(D)])
    K = diagm(K)
    return K
end

## Eliminating $w_1$

We are only interested in $w_2$ -- $w_1$ doesn't enter into the constraint, so we will eliminate it using the KKT conditions.

This means we want to 

* partition $B=$ `G_of_w[1]` into $B_{11}$, $B_{12}$, and $B_{22}$

* partition $b$ into $b_1$ and $b_2$

* solve for $\hat{B}$ and $\hat{b}$.

In [ ]:
function partition_B(G_of_w,Q_of_w) # cheap
    B,b = G_of_w[1],G_of_w[2]
    Q = round(Q_of_w[1])
    i2 = find(diag(Q))
    i1 = setdiff(1:size(Q,1),i2)
    B11,B12,B21,B22 = B[i1,i1],B[i1,i2],B[i2,i1],B[i2,i2]
    b1 = b[i1]
    b2 = b[i2]
    return B11,B12,B21,B22,b1,b2
end

function return_Bhat(B11,B12,B22,b1,b2) # expensive: matrix inversion
    Bhat = B22 - (B12'/B11)*B12
    bhat = b2 - (B12'/B11)*b1
    return round(Bhat,10),bhat
end

## Solution via first-order optimality conditions

We now have

\begin{align*}
&& \min~ w_2^\top \hat{B}w_2 + w_2^\top \hat{b} \\
s.t. && w_2^\top w_2 &= c
\end{align*}

Where $\hat{B}$ is diagonal. When we write the first-order optimality conditions, we obtain a relationship for $\hat{w}_2$ in terms of $v$:

\begin{align}
\hat{w}_{2,i} &= \frac{1}{2}\left(\frac{\hat{b}_i}{v - \hat{B}_{i,i}}\right)
\end{align}

In [ ]:
function find_w(v,D,d)
    if v == 0
        w = float([-d[i]/(D[i,i]) for i in 1:length(d)])
    else
        w = float([d[i]/(v - D[i,i]) for i in 1:length(d)])
    end
    return w
end

Now we can formulate and solve the secular equation. The general form is:

\begin{align}
s(v) &= \sum_i \left(\frac{d_i}{v - D_{i,i}}\right)^2 = c
\end{align}

The following method solves this equation using binary search:

In [ ]:
function solve_secular(D,d,c)
    """ Solve the secular equation via binary search.
    """
    eps = 1e-8
    solutions = Float64[]
    vectors = Array(Vector{Float64},0)
    poles = sort(unique(round(diag(D),10)))
    
    # Each diagonal element is a pole.
    for i in 1:length(poles)
        
        # Head left first:
        high = poles[i]
        if length(poles) == 1
            low = high - high
        elseif i == 1
            low = high - abs(poles[i] - poles[i+1])
        else
            low = high - abs(poles[i] - poles[i-1])/2
        end
        
        # Initialize v:
        v = (high + low)/2
        w = find_w(v,D,d)
        diff = (w'*w)[1] - c
        diff_old = 0
        stall = false
        while abs(diff) > eps
            if diff == diff_old
                stall = true
                break
            end
            if diff > 0
                high = v
            else
                low = v
            end
            v = (high + low)/2
            w = find_w(v,D,d)
            diff_old = diff
            diff = (w'*w)[1] - c
        end
        if !stall
            push!(solutions,v)
            push!(vectors,w)
        end
        
        # Now head right:
        high = poles[i]
        if length(poles) == 1
            low = high + high
        elseif i == length(poles)
            low = high + abs(poles[i] - poles[i-1])
        else
            low = high + abs(poles[i] - poles[i+1])/2
        end
        
        v = (high + low)/2
        w = find_w(v,D,d)
        diff = (w'*w)[1] - c
        diff_old = 0
        stall = false
        while abs(diff) > eps
            if diff == diff_old
                stall = true
                break
            end
            if diff > 0
                high = v
            else
                low = v
            end
            v = (high + low)/2
            w = find_w(v,D,d)
            diff_old = diff
            diff = (w'*w)[1] - c
        end
        if !stall
            push!(solutions,v)
            push!(vectors,w)
        end
    end
    return solutions,vectors
end

## Mapping back

Having found solutions $\hat{w}_2^*$, we need to map back to the original problem space.

In [ ]:
function return_xopt(w2opt,B11,B12,b1,N,U,K,x_star) # expensive: inversion
    w1opt = -B11\(B12*w2opt + b1/2)
    wopt = [w1opt,w2opt]
    xopt = (N*U/K)*wopt + x_star
    return xopt
end

Here is another opportunity for speedup. Apparently inverting $K$ is expensive, even though it is diagonal. The polyalgorithm is terrible.

In [43]:
K = diagm(rand(500))
L = rand(400,500)
@time L/K
@time L*(diagm(1./diag(K)))

elapsed time: 0.01114078 seconds (10088528 bytes allocated)
elapsed time: 0.016562231 seconds (3648320 bytes allocated)


400x500 Array{Float64,2}:
 1.76932   1.19322    0.94956   …  0.110186     0.924682   1.265     
 0.956887  0.212341   0.124174     1.40674      1.14694    0.893     
 1.46273   1.18728    2.36477      1.48218      0.338783   0.822923  
 0.134922  0.247766   0.706611     0.180975     0.0249464  0.6792    
 0.918634  0.379625   2.081        0.567096     0.29876    1.74322   
 0.871984  0.977775   1.74274   …  0.000901539  0.784611   0.762492  
 0.705524  1.23108    2.35021      0.245582     0.536458   0.320749  
 1.37011   0.494233   0.400555     1.93783      0.722335   1.07643   
 0.874345  0.53897    2.46782      1.39553      0.200936   1.6546    
 0.134486  0.885155   0.746886     2.0576       0.866975   1.40457   
 1.16014   0.0696139  2.27619   …  1.43642      0.497229   1.55525   
 0.358806  0.730758   2.49916      0.2303       0.896409   1.07667   
 0.37583   0.928391   0.711274     0.894        0.250464   1.76745   
 ⋮                              ⋱                               

This does seem to have made a difference.

Today I sped up my code by a factor of 21.

In [51]:
7*60./20

21.0